In [21]:
import pandas as pd
import utils
import sklearn
import gensim

# Load Data

In [28]:
#dataset 2 is for text clustering
url_en2 = 'https://github.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/raw/main/Preprocessing/data_clean/en_clean_2.csv'
df_en2 = pd.read_csv(url_en2, sep=',', encoding='utf-8', error_bad_lines=False)

C:\Users\joana\AppData\Local\Temp\ipykernel_22100\2467272692.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_en2 = pd.read_csv(url_en2, sep=',', encoding='utf-8', error_bad_lines=False)


In [29]:
df_en2

,data,player,language,publishedAt
0,footballflorian wirtz goal bayer leverkusen eu...,exequiel palacios,en,2023-02-16T23:56:00Z
1,xasoccer football europa league play second le...,exequiel palacios,en,2023-02-23T20:50:50Z
2,pickworth mailonlineview commentsbayer leverku...,exequiel palacios,en,2023-02-23T20:53:59Z
3,buenos aires world cup winner argentina celebr...,exequiel palacios,en,2023-03-03T16:40:46Z
4,sign insign inthe star editionchange locationt...,exequiel palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
284,australian associated pressview commentsthe so...,piero hincapie,en,2023-03-28T11:24:40Z
285,julien laurens explains choice espn top star w...,piero hincapie,en,2023-03-29T11:35:08Z
286,"futurechelsea rumored beaten ""up dozen"" teams ...",piero hincapié,en,2023-04-27T04:57:02Z
287,man city's alex robertson makes debut aiden o'...,piero hincapié,en,2023-03-24T15:24:08Z


# Trying sklearn tfidfVectorizer

In [46]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
'''info: by default this doesn't use a stopword list, but it does remove accents and transforms the data into lowercase'''
from sklearn.compose import ColumnTransformer

In [47]:
vec_count = CountVectorizer(max_df=0.95, min_df=0.05)
vec = TfidfVectorizer(max_df=0.95, min_df=0.05) #smoothing and inverse document frequency (= use_idf) enabled, to prevent zero devisions and have the real tfidf not only term frequency
vec_term = TfidfVectorizer(max_df=0.95, min_df=0.05,use_idf=False, norm='l1') #only term frequency is disabled

#it should only be performed on the column data, so we use ColumnTransformer to uapply tfidf selectively
columns_to_remain = ['player', 'language', 'publishedat']
column_transformer = ColumnTransformer(
    transformers=[
        ('tfidf', vec, df_en2['data']),
        ('remainder', 'passthrough', columns_to_remain)  # Include the remaining columns as-is
    ])

In [49]:
transformed_data = column_transformer.fit_transform(df_en2)

ValueError: Estimator names conflict with constructor arguments: ['remainder']

In [ ]:
tfidf_count = vec_count.fit_transform(df_en2).toarray()
tfidf = vec.fit_transform(df_en2).toarray()
tfidf_term = vec_term.fit_transform(df_en2).toarray()

In [43]:
print('countvectorizer', tfidf_count)
print('tfidf:', tfidf)
print('term frequency:', tfidf_term)

countvectorizer [[1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]]
tfidf: [[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
term frequency: [[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [44]:
df_tfidf = pd.DataFrame(tfidf, columns=vec.vocabulary_.keys())
df_tfidf.head()

,data,player,language,publishedat
0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0


# Trying gensims tfidfModel

# Comparing the performance of sklearn and gensim